## This notebook should not be relaunched once you have the df_file_final.csv up

The goal of this notebook is to create a **DataFrame** containing all phrases link to the company in each document, but also to add some features (when available) such as Title, Frequency and Count of company name

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm_pandas
from tqdm import tqdm_notebook as tqdm
import extract_text
import text_summarization
import spacy

In [3]:
df_files = pd.read_csv("submission_mapper.csv",delimiter="|")
df_files.head()

,id,filename,company
0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel
1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel
2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel
3,3,businessassociatelist--1-.xlsx,Keppel
4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel


In [4]:
df_files['file_path'] = df_files.apply(lambda x : "files/"+x.company+"/"+x.filename,axis=1)
#files.index = files['file_path']
df_files['sentiment'] = np.nan
df_files.head()

,id,filename,company,file_path,sentiment
0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN
1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN
2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN
3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN
4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN


In [5]:
#tqdm().pandas(desc="Progress:")
#phrases = pd.read_csv("Phrases.csv")
phrases = pd.DataFrame(data=[],columns=["Sentences"])
phrases

,Sentences


In [ ]:
keppelWord = ['keppel corporation', 'keppel capital', 'keppel reit', 'keppel infrastructure','keppel o&m', 'keppel energy', 'keppel offshore', 'keppel dhcs', 'keppel fairway', 'keppel professorship', 'keppel reit', 'keppel group', 'keppel land' , 'keppel singmarine', 'keppel fels', 'keppel shipyard','keppel shipyard' , 'keppel gas','keppel t&t', 'keppel bay', 'keppel telecom','keppel corp','keppel seghers','keppel center','keppel cebu','keppel thai','keppel philippines','keppel tatle','keppel  telecoms','keppel tower']
prudentialWord = ['prudential mutual','prudential plc']

In [1]:
##Code to drop non interesting documents
def doWeDrop(sentences):
    isValid = False
    for phrase in sentences:
        lowered = phrase.lower()
        #print(lowered)
        for expr1 in keppelWord:
            if(expr1 in lowered):
                #print("valid")
                isValid = True
        for expr2 in prudentialWord:
            if(expr2 in lowered):
                #print("valid")
                isValid = True
        
    return not isValid

In [55]:
nlp = spacy.load('en_core_web_sm')
##Fill sentences, meaning extract text and summarize it.
def fillSentences(path,company):
    print(path)
    print(company)  
    try:
        text = extract_text.apply(path)
        if doWeDrop(text):
            return ["-500"]
        important_sents = text_summarization.apply(text,company)
        print(len(important_sents))
        return important_sents
    except:
        print("error")
        return [" "]

In [ ]:
##Dont't relaunch this it is used to fill the DF, it is realllllly long
tqdm().pandas(desc="Progress:")
df_files['sentences'] = ""
phrases["Sentences"] = df_files.progress_apply(lambda row : fillSentences(row["file_path"],row["company"].lower()),axis=1)

In [9]:
##Load sentences into df_files. (Apply was returning a error due to dimension problem so we used .loc)
df_files_new = df_files
for i in range(len(phrases.Sentences)):
    df_files_new.loc[i,'sentences'] = phrases.Sentences[i]


In [11]:
## Save in a csv.
df_files_new.to_csv("safe/df_file_final.csv")
df_files_new

,id,filename,company,file_path,sentiment,sentences
0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN,([('Keppel Capital aims to create value and de...
1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN,([('Company Backg round Keppel is a diversifie...
2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN,([('The new SOCAR-Keppel Shipyard will be deve...
3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN,['-500']
4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN,([('Jurong Shipyard Pte Ltd (Songa Eclipse) Ju...
5,5,sin63964.doc,Keppel,files/Keppel/sin63964.doc,NaN,([('2.3.6 West Keppel Fairway That part of the...
6,6,KOMTechnology%20StrategyJuly03v3.pdf,Keppel,files/Keppel/KOMTechnology%20StrategyJuly03v3.pdf,NaN,"([('Within Keppel O&M, there are three technol..."
7,7,Keppel%20REIT_AR2016.pdf,Keppel,files/Keppel/Keppel%20REIT_AR2016.pdf,NaN,([('employees from across the Keppel Group att...
8,8,pre_13sep2010_n2.pdf,Keppel,files/Keppel/pre_13sep2010_n2.pdf,NaN,"([(""Working experience and occupation(s) durin..."
9,9,Dec06.xls,Keppel,files/Keppel/Dec06.xls,NaN,['-500']


# Below this was code to extract extra feature which is not used, because of the lack of time

In [1]:
#df_files_new['title'] = df_files.apply(lambda row : extract_text.extract_title(row['file_path']),axis=1)

In [2]:
#df_files.head(5)

In [3]:
#df_files_new['occurence'] = df_files.apply(lambda row : extract_text.count_occurence(extract_text.apply(row['file_path']),row['company']),axis=1)

In [4]:
#df_files_new['freq'] = df_files.apply(lambda row : extract_text.frequency_occurence(extract_text.apply(row['file_path']),row['company']),axis=1)

#df_files_new

In [ ]:
#df_files_new.to_csv("df_file_final.csv")